In [1]:
# Dependencies and Setup
import pandas as pd
import os
import numpy as np

In [2]:
# File to Load (Remember to change the path if needed.)
def getPath(folder,file):
    return os.path.join(folder,file)
school_data_to_load = getPath("Resources","schools_complete.csv")
student_data_to_load = getPath("Resources","students_complete.csv")

In [3]:
# Read the School Data and Student Data and store into a Pandas DataFrame
school_data_df = pd.read_csv(school_data_to_load)
student_data_df = pd.read_csv(student_data_to_load)

In [ ]:
#df before replacements
#student_data_df

In [4]:
# Cleaning Student Names and Replacing Substrings in a Python String
# Add each prefix and suffix to remove to a list.
replaceThese = {"Dr. ":"", "Mr. ":"","Ms. ":"", "Mrs. ":"", "Miss ":"",
                     " MD":"", " DDS":"", " DVM":"", " PhD":""}

# Iterate through the words in the "prefixes_suffixes" list and replace them with an empty space, "".
def cleanNames(df,col):
    return df.replace(replaceThese, regex=True, inplace=True)
cleanNames(student_data_df,'student_name')

In [ ]:
# Check names.
student_data_df

In [5]:
#Deliverable 1

#The loc method is used to select all the reading and math scores from 
#the ninth grade at Thomas High School. Inside the loc method, the following are completed:

# 1. A comparison operator is used to retrieve all the rows with Thomas High School 
#   in the "school_name" column of the student_data_df (10 pt).
# 2. A comparison operator is used to retrieve all the rows with the ninth grade 
#   in the "grade" column of the student_data_df (10 pt).
# 3. Logical and comparison operators are used to retrieve all the rows with the 
#   "reading_score" column for Thomas High School ninth graders from the student_data_df (10 pt).
# 4. Logical and comparison operators are used to retrieve all the rows with the 
#   "math_score" column for Thomas High School ninth graders from the student_data_df (10 pt).

df = student_data_df
conditions = (student_data_df.school_name == 'Thomas High School')&(student_data_df.grade == '9th')

for i in {'reading_score','math_score'}:
    df.loc[conditions,i] = np.nan

#The reading and math scores for the ninth graders in Thomas High school are replaced with NaNs (10 pt).
student_data_df.tail(10)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
39160,39160,Katie Weaver,F,11th,Thomas High School,89.0,86.0
39161,39161,April Reyes,F,10th,Thomas High School,70.0,84.0
39162,39162,Derek Weeks,M,12th,Thomas High School,94.0,77.0
39163,39163,John Reese,M,11th,Thomas High School,90.0,75.0
39164,39164,Joseph Anthony,M,9th,Thomas High School,NaN,NaN
39165,39165,Donna Howard,F,12th,Thomas High School,99.0,90.0
39166,39166,Dawn Bell,F,10th,Thomas High School,95.0,70.0
39167,39167,Rebecca Tanner,F,9th,Thomas High School,NaN,NaN
39168,39168,Desiree Kidd,F,10th,Thomas High School,99.0,90.0
39169,39169,Carolyn Jackson,F,11th,Thomas High School,95.0,75.0


In [ ]:
#Deliverable 2

In [6]:
#will use to get student counts
isnaCounts = student_data_df['math_score'].isna().value_counts()

In [7]:
#combined school and student data frames
mergedDF = pd.merge(student_data_df, school_data_df, how="left", on=["school_name", "school_name"])
#stored justed THS ninth grade data
THSninth = df.loc[conditions]

In [8]:
#Storing the passing grade
passed = 70

#Storing passing grade conditions
passedMath = mergedDF[(mergedDF["math_score"] >= passed)]
passedReading = mergedDF[(mergedDF["reading_score"] >= passed)]
passedOverall = mergedDF[(mergedDF["reading_score"] >= passed) & (mergedDF["math_score"] >= passed)]


#functions to use later
def getThis(df,col,pandas_stat):
    return getattr(df[col], pandas_stat)()

def getSeries(df,col):
    return df[col]
    
def getPercent(x,y):
    return x / y * 100

def makeDF(dic):
    return pd.DataFrame(dic)

def formatThis(df,col,how):
    return df[col].map(how.format)

In [9]:
# 1. The district summary DataFrame (3 pt)

#Storing column names in a list
columnList = ["Total Schools", "Total Students", "Total Budget","Average Math Score", 
              "Average Reading Score", "% Passing Math","% Passing Reading", "% Overall Passing"]

#Get values
schoolCount = len(getThis(mergedDF,'school_name','unique'))
totalStudents = isnaCounts[False]
totalBudget = getThis(school_data_df,'budget','sum')
avgMath = getThis(mergedDF,'math_score', 'mean')
avgReading = getThis(mergedDF,'reading_score','mean')
passingMath = getPercent(getThis(passedMath,'Student ID','count'),totalStudents)
passingReading = getPercent(getThis(passedReading,'Student ID','count'),float(totalStudents))
passingOverall = getPercent(getThis(passedOverall,'Student ID','count'),float(totalStudents))

districtVars = [schoolCount, totalStudents, totalBudget, avgMath, avgReading, passingMath, passingReading, passingOverall]

#Make dictionary of column names and values
districtDict = dict(zip(columnList,districtVars))

#Make data frame
districtSummary = makeDF([districtDict])

#Format
districtSummary['Total Students'] = formatThis(districtSummary,'Total Students',"{:,}")
districtSummary["Total Budget"] = formatThis(districtSummary,"Total Budget","${:,.2f}")
for i in columnList[3:]:
    districtSummary[i] = formatThis(districtSummary,i,"{:.1f}")

In [10]:
districtSummary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"38,709","$24,649,428.00",78.9,81.9,74.8,85.7,64.9


In [11]:
# 2. The school summary DataFrame (3 pt)

def groupThis(df,group):
    return df.groupby([group])

#List of column names
perSchoolcolumns = ["School Type","Total Students","Total School Budget",
                    "Per Student Budget","Average Math Score","Average Reading Score",
                    "% Passing Math","% Passing Reading","% Overall Passing"]
#Make groups
bySchoolname = groupThis(mergedDF,"school_name")
passedMathbySchool = getThis(groupThis(passedMath,'school_name'),'student_name','count')
passedReadingbySchool = getThis(groupThis(passedReading,'school_name'),'student_name','count')
passedOverallbySchool = getThis(groupThis(passedOverall,'school_name'),'student_name','count')

# Calculate the total school budget and per capita spending
per_school_budget = getThis(bySchoolname,"budget",'mean')


# Calculate the average test scores.
per_school_math = getThis(bySchoolname,'math_score','mean')
per_school_reading = getThis(bySchoolname,"reading_score",'mean')

#Set index
per_school_types = getSeries(school_data_df.set_index(["school_name"]),"type")
#Students per school
per_school_counts = getSeries(mergedDF,"school_name").value_counts()
#Per student budget
per_school_capita = per_school_budget / per_school_counts

#Get passing percents
per_school_passing_math = getPercent(passedMathbySchool,per_school_counts)
per_school_passing_reading = getPercent(passedReadingbySchool,per_school_counts)
per_overall_passing_percentage = getPercent(passedOverallbySchool,per_school_counts)

#List of values
perSchoolvars = [per_school_types, per_school_counts, per_school_budget, per_school_capita, 
                 per_school_math, per_school_reading, per_school_passing_math, 
                 per_school_passing_reading, per_overall_passing_percentage]

#Make dictionary
perSchooldict = dict(zip(perSchoolcolumns,perSchoolvars))

perSchoolsummary = makeDF(perSchooldict)

#Format
for i in perSchoolcolumns[2:4]:
    perSchoolsummary[i] = formatThis(perSchoolsummary,i,"${:,.2f}")

In [12]:
perSchoolsummary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [13]:
# 2. The school summary DataFrame (3 pt)
# Replace THS values

THS10to12 = mergedDF.dropna()

#THS 10-12 count
THSstudents = THS10to12.loc[THS10to12.school_name == 'Thomas High School']
THScount = float(getThis(THSstudents,'School ID','count'))

#Passing conditions
THSpassedMath = THSstudents[(THSstudents["math_score"] >= passed)]
THSpassedReading = THSstudents[(THSstudents["reading_score"] >= passed)]
THSpassedOverall = THSstudents[(THSstudents["math_score"] >= passed)
                   & (THSstudents["reading_score"] >= passed)]

#Get percents
THSpassing_math_percentage = getPercent(getThis(THSpassedMath,"student_name",'count'),THScount)
THSpassing_reading_percentage = getPercent(getThis(THSpassedReading,"student_name",'count'),THScount) 
THSpassing_overall_percentage = getPercent(getThis(THSpassedOverall,"student_name",'count'),THScount)

#Replace values


perSchoolsummary.loc['Thomas High School',"% Passing Math"] = THSpassing_math_percentage

perSchoolsummary.loc['Thomas High School',"% Passing Reading"] = THSpassing_reading_percentage

perSchoolsummary.loc['Thomas High School',"% Overall Passing"] = THSpassing_overall_percentage

In [14]:
perSchoolsummary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [15]:
# 3. The top 5 performing schools, based on the overall passing rate (2 pt)
topSchools = perSchoolsummary.sort_values(["% Overall Passing"], ascending=False)

In [16]:
topSchools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.350937,83.896082,93.185690,97.018739,90.630324
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [17]:
# 4. The bottom 5 performing schools, based on the overall passing rate (2 pt)
bottomSchools = perSchoolsummary.sort_values(["% Overall Passing"], ascending=True)

In [18]:
bottomSchools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [19]:
# Create a Series of scores by grade levels using conditionals.
gradeLevels = ['9th','10th','11th','12th']

grade = [THS10to12[THS10to12['grade'] == i] for i in gradeLevels]

# 5. The average math score for each grade level from each school (3 pt)
mathBygrade = [getThis(groupThis(i,'school_name'),'math_score','mean') for i in grade]

# 6. The average reading score for each grade level from each school (3 pt)
readingBygrade = [getThis(groupThis(i,'school_name'),'reading_score','mean') for i in grade]

#Save to data frames
math_scores_by_grade = makeDF(dict(zip(gradeLevels,mathBygrade)))
reading_scores_by_grade = makeDF(dict(zip(gradeLevels,readingBygrade)))

#Format
math_scores_by_grade.index.name = None
reading_scores_by_grade.index.name = None

for i in gradeLevels:
    math_scores_by_grade[i] = formatThis(math_scores_by_grade,i,"{:.1f}")
    reading_scores_by_grade[i] = formatThis(reading_scores_by_grade,i,"{:.1f}")

In [20]:
math_scores_by_grade

,9th,10th,11th,12th
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4
Hernandez High School,77.4,77.3,77.1,77.2
Holden High School,83.8,83.4,85.0,82.9
Huang High School,77.0,75.9,76.4,77.2
Johnson High School,77.2,76.7,77.5,76.9
Pena High School,83.6,83.4,84.3,84.1


In [21]:
reading_scores_by_grade

,9th,10th,11th,12th
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0
Hernandez High School,80.9,80.7,81.4,80.9
Holden High School,83.7,83.3,83.8,84.7
Huang High School,81.3,81.5,81.4,80.3
Johnson High School,81.3,80.8,80.6,81.2
Pena High School,83.8,83.6,84.3,84.6


In [24]:
# 7. The scores by school spending per student (3 pt)
spending_bins = [0, 585, 630, 645, 675]
group_names = ["<$584", "$585-629", "$630-644", "$645-675"]
spendingColumns = ["Average Math Score", "Average Reading Score", "% Passing Math",
                   "% Passing Reading", "% Overall Passing"]

# Categorize spending based on the bins.
perSchoolsummary["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=group_names)


# Calculate averages for the desired columns. 
spendingValues = [getThis(groupThis(perSchoolsummary,'Spending Ranges (Per Student)'),i,'mean') for i in spendingColumns]

#Make data frame
spendingSummary = makeDF(dict(zip(spendingColumns,spendingValues)))

#Format
for i in spendingColumns[0:2]:
    spendingSummary[i] = formatThis(spendingSummary,i,"{:.1f}")
for i in spendingColumns[2:]:
    spendingSummary[i] = formatThis(spendingSummary,i,"{:.0f}")
    

In [25]:
spendingSummary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.5,83.9,93,97,90
$585-629,81.9,83.2,87,93,81
$630-644,78.5,81.6,73,84,63
$645-675,77.0,81.0,66,81,54


In [26]:
# 8. The scores by school size (3 pt)

# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

sizeColumns = ["Average Math Score", "Average Reading Score", "% Passing Math",
               "% Passing Reading", "% Overall Passing"]

# Categorize spending based on the bins.
perSchoolsummary["School Size"] = pd.cut(perSchoolsummary["Total Students"], size_bins, labels=group_names)

# Calculate averages for the desired columns. 
sizeValues = [getThis(groupThis(perSchoolsummary,'School Size'),i,'mean') for i in sizeColumns]

#Make data frame
sizeSummary = makeDF(dict(zip(sizeColumns,sizeValues)))

#Format
for i in sizeColumns[0:2]:
    sizeSummary[i] = formatThis(sizeSummary,i,"{:.1f}")
for i in sizeColumns[2:]:
    sizeSummary[i] = formatThis(sizeSummary,i,"{:.0f}")

In [27]:
sizeSummary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.8,83.9,94,96,90
Medium (1000-2000),83.4,83.9,94,97,91
Large (2000-5000),77.7,81.3,70,83,58


In [29]:
# 9. The scores by school type (3 pt)

typeColumns = ["Average Math Score", "Average Reading Score", "% Passing Math",
               "% Passing Reading", "% Overall Passing"]

typeValues = [getThis(groupThis(perSchoolsummary,'School Type'),i,'mean') for i in typeColumns]

#Make data frame
typeSummary = makeDF(dict(zip(typeColumns,typeValues)))

#Format
for i in sizeColumns[0:2]:
    typeSummary[i] = formatThis(typeSummary,i,"{:.1f}")
for i in sizeColumns[2:]:
    typeSummary[i] = formatThis(typeSummary,i,"{:.0f}")

In [30]:
typeSummary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.5,83.9,94,97,90
District,77.0,81.0,67,81,54
